In [1]:
# Carrega as variáveis de embiente usadas pelo notebook. Estas variáveis devem ser criadas no arquivo .env, localizada no mesmo diretório do notebook
# As seguintes variáveis devem ser criadas:
# - export STR_CON=<string de conexão com o banco de dados relacional>
# - export RAW_DATA_PATH=<caminho onde os arquivos contendo o histórico de preço de cada ativo será armazenado>
# - export DATASET_PATH=<caminho onde os arquivos contendo o histórico de preço e os indicadores técnicos serão armazenados>
# - export MODELS_PATH=<caminnho onde os modelos treinados serão armazenados>
# - export TRAIN_DATASET=<caminho onde os datasets estandarizados e formatados para o treinamento serão armazenados>
%load_ext dotenv
%dotenv

# Garante que os scripts Python sejam recarregados em cada execução do notebook.
%load_ext autoreload
%autoreload 2

## Objetivo do Nobook 
<p>O objetivo deste notebook é extrair o histórico de preços de ações de banco de dados relacional, acrescentar os indicadores técnicos, rentabilidade e label de classificação e gravá-los no formato <b>parquet</b> para que sejam usados no processo de treinamento do modelo de machine learning. O primeiro passo é obter os dados históricos de negociação de cada ativo gravá-los como arquivos individuais em disco. Estes arquivos devem conter as seguintes colunas:</p>
<ul>
<li>ticker - Código de negociação do ativo</li>
<li>dt_price - Data de negociação</li>
<li>open - Preço de abertura no dia</li>
<li>close - Preço de fechameto no dia</li>
<li>high - Preço máximo no dia</li>
<li>low - Preço mínimo no dia</li>
<li>volume - Volume financeiro negociado do ativo no dia</li>
</ul>
<p>É fundamental que os arquivos contenham exatamente estas colunas, pois a biblioteca TA-Lib (utilizada para calcularos indicadores técnicos) espera estes nomes como parâmetros de entrada para suas funções.</p>
<p><b>Exemplo</b>:</p>
<p>Lista de ativos recuperados do banco de dados: [PETR4, ITUB4, VALE3, ABEV3]</p>
<p>Arquivos gerados:</p>
<ul>
<li> PETR4.parquet </li>
<li> ITUB4.parquet </li>
<li> VALE3.parquet </li>
<li> ABEV3.parquet </li>
</ul>
<p>Uma vez gerados os arquivos <b>parquet</b> com os dados históricos de preço dos ativos, os indicadores técnicos serão calculados de acordo com as estratégias definias no arquivo de configuração <b>strategies.json</b> e a rentabilidade em X dias futuros calculada. Esta rentabilidade será usada para definir o label que será usado para classificar o ativo.</p>

### Imports

In [6]:
import os
import pandas as pd
import json
from data_preparation import PreProcess, PROFTABILITY_TYPE
from db_access import StockHistory, ExportToParquet, ExportToPickle
from sklearn.preprocessing import MinMaxScaler

pd.options.display.max_columns = 200

### Geração dos arquivos apenas com o a série histórica de preços

In [3]:
# Obtem o historico de precos dos ativos 
str_conn = os.environ.get("STR_CON")
raw_file_path = os.environ.get("RAW_DATA_PATH")
data_file_path = os.environ.get("DATASET_PATH")
exporter = ExportToParquet()
dbo = StockHistory(str_conn)
df = dbo.select()
df["dt_price"] = df["dt_price"].astype('datetime64[ns]')
# Grava os arquivos com o histórico de preços em formato parquet em um diretório local
print(f"Gerando arquivos de preço")
for stock in df['ticker'].unique():
    df_aux = df.loc[df['ticker'] == stock]
    exporter.export(df_aux, raw_file_path, stock)
print(f"Arquivos de preço gerados com sucesso")

### Cálculo dos indicadores técnicos, e rentabilidade dos ativos
<p>O segundo passo da preparação dos datasets consiste em incluir os indicadores técnicos ao histórico de preço de cada ativo de acordo com as estratégias de negociação definidas no arquivo de configuração, calcular a rentabilidade do ativo X dias para frente e usar esta rentabilidade para definir o label de classificação do ativo (1 se o ativo se valorizou mais do que Y% X dias após a série de dados do registro do dataset ou 0 caso contrário).</p> 
<p>Uma estratégia de negociação nada mais é do que a combinação de indicadores técnicos que serão usados como features em modelos de machine learning. O ideia é realizar diversos experimentos com diferentes estratégias para encontrar combinações que ajudem melhorar a acurácia do modelo na classificação dos ativos com maior potencial de rentabilidade no período informado.</p>
<p>Para automatizar o processo, as estratégias de negociação são configuradas em um arquivo json com a seguinte estrutura:</p>
<code>
<pre>
    "&lt;nome da estrategia&gt;":{
        "description": "&lt;Descrição da estratégia&gt;",
        "historic_period": &lt;Número inteiro que irá definir o total dias usados pelo modelo. Por exemplo, o modelo receberá um histórico de preço de 
                            60 dias para classificar o ativo&gt;,
        "profit_period": &lt;Número inteiro que definirá o período usado para calcular a rentabilidade do ativo (será usado para definir seu label).
                          Por exemplo, o modelo reberá 60 dias de cotação para classificar o ativo, se este parâmetro for setado para 10, a rentablidade 
                          usada para definir o label será calclada como: 
                          preço 10 dias após a o último dia da série de entrada/preço do último dia da série de entrada - 1&gt;,
        "shift_days": &lt;Número inteiro que define quantos dias serão "pulados" durante a geração das series temporais de preços. Por exemplo, um modelo
                       poderá ser treinado com um histórico de preços de 30 dias, a primeira série histórica será do dia 01 até o dia 30 do mês, se
                       shift_days for igual a 5 dias, a segunda série será do dia 6 (cinco dias após o início da série anterior) até o dia 6 mais 30
                       dias&gt;,
        "profit": &lt;Rentabilidade usada para definir o label de classificação de cada registro enviado para o modelo durante o treinamento. Por exemplo,
                   Se o objetivo do modelo for identificar ativos que possuam rentabiliade superior a 5% em um período de 20 dias, o valor atribuído para
                   este parâmetro deve ser 0.05&gt;,
        "functions": { -- Lista de funções que fazem parte da estratégia de negociação
            "&lt;nome "fantasia" da função&gt;":{
                "function": "&lt;nome da função na biblioteca TA-Lib, será usado para chamar a função no código&gt;",
                "params":{ -- Lista de parâmetros da função (parâmetros relacionados ao preço do ativo não devem ser listados aqui)
                    "&lt;nome da parâmetro&gt;": &lt;valor&gt;
                }                
            }
        }
    }
</pre>
</code>
<p>Exemplo de configuração:</p>
<p>Neste exemplo, a estratégia considera 90 dias de histórico de preço do ativo para classificar ativos que tenham rentabilidade superior a 5% em um período de 22 dias e para fazer isso, utiliza além do histórico de cotações os indicadores: Média móvel exponencial de 7 e 21 dias e o OBV.</p>
<code>
<pre>
{
    "EMA_7_21_OBV":{
        "description": "Combinacao de 2 medias moveis (7 e 21 dias) e OBV ",
        "historic_period": 90,
        "profit_period": 22,
        "shift_days": 5,
        "profit": 0.05,
        "functions": {
            "EMA_7":{
                "function": "EMA",
                "params":{
                    "timeperiod": 7
                }
            },
            "EMA_21": {
                "function": "EMA",
                "params":{
                    "timeperiod": 21
                }            
            },
            "OBV":{
                "function": "OBV"                  
            }
        }
    }
}
</pre>
</code>
<p>Após calcular todos os indicadores técnicos, o dataset será formatado para que as séries históricas possam ser passadas pelos modelo. Para isso, cada uma das colunas do DataFrame será transposta de acordo com os valores configrados no arquivo <b>strategies.json</b>. Por exemplo, se o parâmetro historic_period estiver configurado para 30, 30 registros de cada coluna se transformarão em 30 colunas em um único registro. Após transpor todas as colunas, o sistema irá "pular" <b>shift_days</b> registros do DataFrame com a série histórica e repetir o processo de transposição, excluíndo os <b>shift_days</b> valores mais antidos de cada coluna e incluíndo novos valores.</p>
<p>Por fim, a rentabilidade do ativo é calculada <b>profit_period</b> dias após o último dia da série do registro e se seu valor superior ao limite definido no parâmetro <b>profit</b> o label do registro será definido como 1, caso contrário 0.</p>
<pre>
Exemplo:<br/>
    * historic_period = 45
    * profit_period = 15
    * shift_days = 5
    * profit = 0.05
    
Cada registro do dataset de treinamento terá 45 colunas para cada coluna do Dataframe original.
O período de rentabilidade será calculado considerando as cotações do 46º e 60º (45 + 15) valores.
Todos ativos que possuirem rentabilidade maior ou igual a 5% serão marcados com 1 no valor do label, os demais com 0.
A primeira linha do novo dataset, conterá todos os valores compreendidos entre as linhas 1 e 45 do DataFrame original, a segunda linha conterá todos os 
valores entre as linhas 6 e 50 do DataFrame original.
</pre>
<p>Ao final do processo, arquivos parquet serão gravados contendo os valores calculados e formatados. Cada estratégia irá gerar um diretório contendo os arquivos com os dados e existirá um arquivo por ativo processado.</p>

In [3]:
pre_process = PreProcess()
exporter = ExportToParquet()

raw_file_path = os.environ.get("RAW_DATA_PATH")
data_file_path = os.environ.get("DATASET_PATH")

path_content = os.listdir(raw_file_path)
# Filtra os arquivos parquet do diretório
path_content = [file for file in path_content if file.endswith(".parquet")]

for file in path_content:
    print(f"Processando arquivo: {raw_file_path}/{file}")
    df_raw = pd.read_parquet(os.path.join(raw_file_path, file))
    df_raw = df_raw.set_index("dt_price", drop=False)

    # Calcula os indicadores técnicos definidos nas estratégias para cada ativo
    for strategy_name, strategy, df_tech in pre_process.calculate_strategy("strategies.json", df_raw):
        window_size = strategy["historic_period"]
        shift = strategy["shift_days"]
        profit_period = strategy["profit_period"]
        profit = strategy["profit"]

        # Remove os valores nulos do DataFrame
        df_tech.dropna(inplace=True)

        # Faz a transposição dos valores para montar o data set de treino
        df_transp = pre_process.transpose_columns(df_tech, window_size, shift)
        del(df_tech)
        # Calcula a rentabilidade do período futuro
        df_transp["proftability"] = [pre_process.calculate_proftability(df_raw, element, profit_period, PROFTABILITY_TYPE.LOG) for element in df_transp["end_dt_price"]]
        # Define o label de cada registro do data set de treino
        df_transp["label"] = [1 if val else 0 for val in df_transp["proftability"] >= profit]
        # Exporta o data set como um arquivo parquet
        exporter.export(df_transp, os.path.join(data_file_path, strategy_name), file.split('.')[0])

Processando arquivo: data/raw/CPFE3.parquet
Processing strategy: Combinacao de 2 medias moveis (7 e 21 dias) e OBV 
Calculating EMA_7...
Calculating EMA_21...
Calculating OBV...
Processing strategy: Combinacao de Bolinger Bands (14, 2, 2), ADX e OBV 
Calculating BBANDS_14_2_2_EMA...
Calculating ADX...
Calculating OBV...
Processing strategy: Combinacao Preço Médio de 14 dias, Média Móvel Exponencial de 14 dias e OBV 
Calculating MIDPRICE_14...
Calculating KAMA_14...
Calculating OBV...
Processando arquivo: data/raw/PMAM3.parquet
Processing strategy: Combinacao de 2 medias moveis (7 e 21 dias) e OBV 
Calculating EMA_7...
Calculating EMA_21...
Calculating OBV...
Processing strategy: Combinacao de Bolinger Bands (14, 2, 2), ADX e OBV 
Calculating BBANDS_14_2_2_EMA...
Calculating ADX...
Calculating OBV...
Processing strategy: Combinacao Preço Médio de 14 dias, Média Móvel Exponencial de 14 dias e OBV 
Calculating MIDPRICE_14...
Calculating KAMA_14...
Calculating OBV...
Processando arquivo: d

### Criação dos datasets usados no treinamento
<p>As últimas etapas da perparação de dados é a divisão do dataset em dois subconjuntos (treinamento e validação) e o redimensionamento dos valores, para que todos os números tenham a mesma ordem de grandeza (números entre 0 e 1). O processo de redimensionamento dos valores é feito por ativo, e os datasets resultantes são concatenados, gerando um dataset completo que será divido em 2 (treino e teste) e gravado como arquivo parquet para ser usado no treinamento do modelo.</p>

In [7]:
pre_process = PreProcess()
pre_process.create_train_test_dataset("strategies.json", 0.15, 42, MinMaxScaler())

Processando arquivo CPFE3.parquet na estrategia EMA_7_21_OBV
Processando arquivo PMAM3.parquet na estrategia EMA_7_21_OBV
Processando arquivo BEEF3.parquet na estrategia EMA_7_21_OBV
Processando arquivo LOGG3.parquet na estrategia EMA_7_21_OBV
Processando arquivo VIVT3.parquet na estrategia EMA_7_21_OBV
Processando arquivo PLPL3.parquet na estrategia EMA_7_21_OBV
Processando arquivo KEPL3.parquet na estrategia EMA_7_21_OBV
Processando arquivo ROMI3.parquet na estrategia EMA_7_21_OBV
Processando arquivo HAPV3.parquet na estrategia EMA_7_21_OBV
Processando arquivo ARZZ3.parquet na estrategia EMA_7_21_OBV
Processando arquivo SOMA3.parquet na estrategia EMA_7_21_OBV
Processando arquivo PETR3.parquet na estrategia EMA_7_21_OBV
Processando arquivo TRPL4.parquet na estrategia EMA_7_21_OBV
Processando arquivo AGRO3.parquet na estrategia EMA_7_21_OBV
Processando arquivo VAMO3.parquet na estrategia EMA_7_21_OBV
Processando arquivo PSSA3.parquet na estrategia EMA_7_21_OBV
Processando arquivo MRFG

In [28]:
df_test = pd.read_parquet("data/train_dataset/EMA_7_21_OBV/test_data.parquet")
df_test.head(10)

,close_0,close_1,close_2,close_3,close_4,close_5,close_6,close_7,close_8,close_9,close_10,close_11,close_12,close_13,close_14,close_15,close_16,close_17,close_18,close_19,close_20,close_21,close_22,close_23,close_24,close_25,close_26,close_27,close_28,close_29,close_30,close_31,close_32,close_33,close_34,close_35,close_36,close_37,close_38,close_39,close_40,close_41,close_42,close_43,close_44,volume_0,volume_1,volume_2,volume_3,volume_4,volume_5,volume_6,volume_7,volume_8,volume_9,volume_10,volume_11,volume_12,volume_13,volume_14,volume_15,volume_16,volume_17,volume_18,volume_19,volume_20,volume_21,volume_22,volume_23,volume_24,volume_25,volume_26,volume_27,volume_28,volume_29,volume_30,volume_31,volume_32,volume_33,volume_34,volume_35,volume_36,volume_37,volume_38,volume_39,volume_40,volume_41,volume_42,volume_43,volume_44,ema_7_0,ema_7_1,ema_7_2,ema_7_3,ema_7_4,ema_7_5,ema_7_6,ema_7_7,ema_7_8,ema_7_9,...,ema_7_36,ema_7_37,ema_7_38,ema_7_39,ema_7_40,ema_7_41,ema_7_42,ema_7_43,ema_7_44,ema_21_0,ema_21_1,ema_21_2,ema_21_3,ema_21_4,ema_21_5,ema_21_6,ema_21_7,ema_21_8,ema_21_9,ema_21_10,ema_21_11,ema_21_12,ema_21_13,ema_21_14,ema_21_15,ema_21_16,ema_21_17,ema_21_18,ema_21_19,ema_21_20,ema_21_21,ema_21_22,ema_21_23,ema_21_24,ema_21_25,ema_21_26,ema_21_27,ema_21_28,ema_21_29,ema_21_30,ema_21_31,ema_21_32,ema_21_33,ema_21_34,ema_21_35,ema_21_36,ema_21_37,ema_21_38,ema_21_39,ema_21_40,ema_21_41,ema_21_42,ema_21_43,ema_21_44,obv_0,obv_1,obv_2,obv_3,obv_4,obv_5,obv_6,obv_7,obv_8,obv_9,obv_10,obv_11,obv_12,obv_13,obv_14,obv_15,obv_16,obv_17,obv_18,obv_19,obv_20,obv_21,obv_22,obv_23,obv_24,obv_25,obv_26,obv_27,obv_28,obv_29,obv_30,obv_31,obv_32,obv_33,obv_34,obv_35,obv_36,obv_37,obv_38,obv_39,obv_40,obv_41,obv_42,obv_43,obv_44,label
0,0.188119,0.174971,0.182612,0.181818,0.179281,0.191566,0.207518,0.211440,0.205261,0.206422,0.239604,0.229365,0.213018,0.233161,0.228211,0.229307,0.244580,0.253381,0.261576,0.272554,0.266265,0.266641,0.250893,0.244848,0.253868,0.273896,0.272122,0.265089,0.332403,0.322248,0.325545,0.334348,0.337278,0.337983,0.329511,0.343762,0.334348,0.343195,0.337186,0.339601,0.342574,0.336630,0.349507,0.345955,0.338066,0.032934,0.026948,0.119824,0.009470,0.158528,0.069521,0.048067,0.282249,0.038659,0.066735,0.129681,0.058294,0.120677,0.105537,0.086739,0.052738,0.052370,0.202904,0.052375,0.143544,0.088708,0.593553,0.367234,0.049319,0.133463,0.085285,0.043784,0.242509,0.265038,0.496804,0.053171,0.261297,0.323573,0.044910,0.150922,0.060983,0.048471,0.141038,0.078768,0.130238,0.121939,0.038962,0.116930,0.020180,0.108703,0.194830,0.189681,0.188015,0.182339,0.184433,0.186631,0.191986,0.198223,0.196663,0.198238,...,0.330660,0.334487,0.333142,0.337325,0.338649,0.338125,0.341679,0.340802,0.342677,0.198030,0.194194,0.193462,0.190877,0.190230,0.188210,0.187931,0.190639,0.190755,0.192803,0.195387,0.197188,0.199186,0.201164,0.204466,0.206341,0.209026,0.213816,0.216852,0.221691,0.223768,0.225851,0.228648,0.228753,0.231292,0.233280,0.235000,0.238452,0.246250,0.254949,0.262161,0.269639,0.276444,0.281317,0.287159,0.291376,0.295159,0.300805,0.303634,0.306864,0.308792,0.310604,0.315083,0.317234,0.321224,0.059660,0.059842,0.049750,0.056995,0.061686,0.059747,0.058854,0.064173,0.072907,0.080867,0.077974,0.065059,0.055846,0.068280,0.083442,0.068100,0.072043,0.071156,0.085830,0.103720,0.103436,0.086691,0.072171,0.075271,0.092668,0.092117,0.095338,0.095359,0.132086,0.122926,0.107224,0.110703,0.111971,0.115500,0.119555,0.103070,0.104808,0.096087,0.112097,0.116683,0.121622,0.112288,0.109598,0.115691,0.121588,1.0
1,0.668911,0.629517,0.625248,0.600808,0.601682,0.640161,0.649569,0.642209,0.609406,0.597859,0.649505,0.639026,0.660750,0.666002,0.666284,0.718812,0.671358,0.738663,0.732738,0.677752,0.687952,0.691073,0.675665,0.675556,0.686236,0.693976,0.669930,0.697830,0.722200,0.681575,0.686733,0.650057,0.688757,0.682344,0.669725,0.692673,0.689996,0.707692,0.698685,0.678818,0.691089,0.672119,0.671400,0.678358,0.675748,0.105462,0.048365,0.351893,0.094918,0.21